In [1]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

#options = webdriver.ChromeOptions()
#options.add_argument('--ignore-certificate-errors')
#options.add_argument('--incognito')
#options.add_argument('--headless')
#options.add_argument("--disable-blink-features")
#options.add_argument("--disable-blink-features=AutomationControlled")

#driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

In [ ]:
# creates a datafram with all of the needed columns including "title", "author", etc.
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'director': pd.Series(dtype='str'),
                   'release_year': pd.Series(dtype='str'),
                   'length': pd.Series(dtype='str'),
                   'genre': pd.Series(dtype='str'),
                   'imdb_score': pd.Series(dtype='float'),
                   'rating' : pd.Series(dtype='str'),
                   'gross_US_CA' : pd.Series(dtype='str'),
                   'num_votes' : pd.Series(dtype='int')
})

print(df)

pageNum = 1

#specifies the website to scrape and use selenium
#driver.get("https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start={}&ref_=adv_nxt".format(pageNum))

#url = "https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start={}&ref_=adv_nxt".format(pageNum)

i = 0
while True:
    #try:
        # waits until the head-block class is displayed
    #    WebDriverWait(driver, 1).until(
    #        lambda s: s.find_element(By.CLASS_NAME, 'article').is_displayed()
    #    )
    #except TimeoutException:
    #    break

    url = "https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start={}".format(pageNum)
    sesh = requests.Session()
    response = sesh.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)", "Referer": "http://example.com"}, timeout = 5)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    
    
    if (i % 99) == 0:
        time.sleep(1)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("div", class_="lister-item-content")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("h3", class_="lister-item-header").find("a").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_director = item_info.find("p", class_="").find("a").get_text(strip=True)
        except:
            movie_director = pd.NA
        try:
            movie_release = item_info.find("h3", class_="lister-item-header").find("span", class_="lister-item-year text-muted unbold").get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_length = item_info.find("p", class_="text-muted").find("span", class_="runtime").get_text(strip=True)
        except:
            movie_length = pd.NA
        try:
            movie_genre = item_info.find("p", class_="text-muted").find("span", class_="genre").get_text(strip=True)
        except:
            movie_genre = pd.NA
        try:
            movie_score = item_info.find("div", class_="ratings-bar").find("div", class_="inline-block ratings-imdb-rating").find("strong").get_text(strip=True)
        except:
            movie_score = pd.NA
        try:
            movie_rating = item_info.find("p", class_="text-muted").find("span", class_="certificate").get_text(strip=True)
        except:
            movie_rating = pd.NA
        try:
            movie_gross = item_info.find("p", class_="sort-num_votes-visible").find_all("span")[4].get_text(strip=True)
        except:
            movie_gross = pd.NA
        try:
            movie_votes = item_info.find("p", class_="sort-num_votes-visible").find_all("span")[1].get_text(strip=True)
        except:
            movie_votes = pd.NA
        
        #since these values are sometimes non-existent, I use a try-except block
        #to set the value to NaN if there is an error (meaning the value doesn't exist)

        
        #adds each of these values as a new row to the dataframe
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'director': movie_director,
                       'release_year': movie_release,
                       'length': movie_length,
                       'genre' : movie_genre,
                       'imdb_score' : movie_score,
                       'rating' : movie_rating,
                       'gross_US_CA' : movie_gross,
                       'num_votes' : movie_votes

                                      
                      }).to_frame().T], ignore_index=True)
        #df.to_csv('imdb_movies.csv')

    #makes sure the next button exists before trying to click it
    #try:
    #    load_more = driver.find_element(By.LINK_TEXT,'Next »')
    #   if load_more: driver.execute_script("arguments[0].click();", load_more)
    #except: pass
                       

    #after each page it increments i by 1, once i is greater than 33 we have reached
    #the last page so it breaks out of the while statement
    pageNum += 50
    i += 1
    if (i % 99) == 0:
        df.to_csv('imdb_movies_no_selenium.csv')
        print(pageNum)
    if i > 10420:
        break

#saves the dataframe as a csv file
df.to_csv('imdb_movies_no_selenium.csv')
df



Empty DataFrame
Columns: [title, director, release_year, length, genre, imdb_score, rating, gross_US_CA, num_votes]
Index: []
4951
9901
14851
19801
24751
29701
34651
39601
44551
49501
54451
59401


In [6]:
df2 = pd.read_csv("imdb_movies.csv",index_col=0)
df2

,title,director,release_year,length,genre,imdb_score,rating,gross_US_CA
0,The Shawshank Redemption,Frank Darabont,(1994),142 min,Drama,9.3,R,$28.34M
1,The Dark Knight,Christopher Nolan,(2008),152 min,"Action, Crime, Drama",9.0,PG-13,$534.86M
2,Inception,Christopher Nolan,(2010),148 min,"Action, Adventure, Sci-Fi",8.8,PG-13,$292.58M
3,Fight Club,David Fincher,(1999),139 min,Drama,8.8,R,$37.03M
4,Forrest Gump,Robert Zemeckis,(1994),142 min,"Drama, Romance",8.8,PG-13,$330.25M
...,...,...,...,...,...,...,...,...
51544,As in Heaven,Tea Lindeburg,(2021),86 min,Drama,6.1,NaN,545
51545,Drib,Kristoffer Borgli,(2017),88 min,"Comedy, Drama",6.3,TV-MA,545
51546,The Butterfly Tree,Priscilla Cameron,(2017),97 min,Drama,5.5,TV-14,545
51547,Your Move,Luke Goss,(2017),96 min,"Crime, Thriller",3.8,NaN,545


In [ ]:
df2_no_NaN = df2.dropna()

df2_no_NaN['release_year'] = df2_no_NaN['current_price'].str[1:-1]
df2_no_NaN['gross_US_CA'] = df2_no_NaN['gross_US_CA'].str[1:-1]
df2_no_NaN['length'] = df2_no_NaN['length'].str[:-3]
df2_no_NaN